In [1]:
import mesa
import math
from enum import Enum
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
ITERATIONS = 100000
%matplotlib inline

echo_party is the majority party. To get liberal majority results. Run with echo_party = 'liberal'. For conservatiev results, run with echo_party = 'conservative'

In [ ]:
echo_party = 'liberal'

In [188]:
# load in the logistic regression model to map morals to emotions
lrmodel = pickle.load(open('./moral2emotelr.pkl', 'rb'))
moral_categories = ['purity', 'authority', 'fairness', 'degradation', 'care', 
                    'loyalty', 'subversion', 'cheating', 'harm', 'betrayal']


In [211]:
lrmodel.coef_

array([[ 2.14834134,  0.37811929,  1.58736116, -1.36935101,  1.95219789,
         1.8168247 , -1.1393514 , -2.13577438, -2.34801116, -0.9017263 ]])

In [189]:
## generate graphs and centrality statistics at the beginning of each sim.
class GraphHolder:
    def __init__(
        self,
        num_nodes=10, 
        avg_node_degree=3, 
        iter = ITERATIONS,
    ):
        self.G_ = []
        self.degree_ = []
        self.eigen_ = []
        self.betweenness_ = []
        
        for _ in range(iter):
            G = nx.erdos_renyi_graph(n=num_nodes, p=1/avg_node_degree)
            #G = nx.complete_graph(num_nodes)
            self.G_.append(G)
            deg = nx.degree_centrality(G)
            # networkx degree normalizes degree by dividing by |V|-1. Undoing that normalization:
            degree_renorm = {key:(val*(num_nodes - 1)) for key, val in deg.items()}
            self.degree_.append(degree_renorm)
            #self.eigen_.append(nx.eigenvector_centrality(G))
            #self.betweenness_.append(nx.betweenness_centrality(G))
            
        self.G = {idx: i for idx, i in enumerate(self.G_)}
        self.degree = {idx: i for idx, i in enumerate(self.degree_)}
        #self.eigen = {idx: i for idx, i in enumerate(self.eigen_)}
        #self.betweenness = {idx: i for idx, i in enumerate(self.betweenness_)}
    #self.degre

class State(Enum):
    READ = 1
    UNREAD = 0

In [190]:
GH = GraphHolder()

## Construct the Agent Class

In [200]:
class EmoAgent(mesa.Agent):
    def __init__(
        self,
        unique_id,
        model,
        read_state,
        initial_state,
        degree_centrality,
        initial_message,
        party,
    ):
        super().__init__(unique_id, model)

        self.initial_state = initial_state
        self.current_state = initial_state
        self.state_cache = list(initial_state)
        self.degree = degree_centrality
        self.susceptibility = 1/(degree_centrality+1)
        self.read_state = read_state
        self.initial_message = initial_message
        self.party = party
    
    def look_at_neighbors(self):
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        read_neighbors = [
            agent
            for agent in self.model.grid.get_cell_list_contents(neighbors_nodes)
            if agent.read_state is State.READ
        ]
        
        comments = [a.current_state for a in read_neighbors]
        if len(comments) == 0:
            mean_comments = np.NAN
        elif len(comments) == 1:
            mean_comments = comments[0]
        else:
            mean_comments = np.mean(comments, axis = 0)
        return mean_comments                    
            
    def read_messages_and_post(self):
        mean_comments = self.look_at_neighbors()
        if mean_comments is np.NAN:
            self.current_state = np.tanh(self.initial_state + (self.susceptibility*self.initial_message)).reshape(1,10)
            self.read_state = State.READ
        else:
            self.current_state = np.tanh(self.initial_state + (self.susceptibility*self.initial_message + mean_comments)).reshape(1,10)
            self.read_state = State.READ
        self.state_cache.append(self.current_state)
    
    def step(self):
        self.read_messages_and_post()

## Helper functions and the model

In [201]:
def opposition_party(echo_party):
    if echo_party == 'conservative':
        return 'liberal'
    elif echo_party == 'liberal':
        return 'conservative'

def gen_political_agent(echo_party):
    if echo_party == 'conservative':
        # Rep - higher authority & Purity
        poli_init = np.concatenate([np.random.uniform(low=0.5, high=1, size=(1)), # purity
                    np.random.uniform(low=0.5, high =1, size= (1)), #authority
                    np.random.uniform(low=0, high=1, size=(1)), #fairness
                    np.random.uniform(low=0.5, high=1, size=(1)), # degradation
                    np.random.uniform(low=0, high=1, size=(1)), # care
                    np.random.uniform(low=0, high=1, size=(1)), # loyalty
                    np.random.uniform(low=0.5, high=1, size=(1)), # subversion
                    np.random.uniform(low=0, high=1, size=(1)), # cheating
                    np.random.uniform(low=0, high=1, size=(1)), # harm
                    np.random.uniform(low=0, high=1, size=(1))]).reshape(1,10) #betrayal
    elif echo_party == 'liberal':
        # Dem: higher fairness/harm & fairness/cheating
        poli_init = np.concatenate([np.random.uniform(low=0, high=1, size=(1)), # purity
                    np.random.uniform(low=0, high =1, size= (1)), #authority
                    np.random.uniform(low=0.5, high=1, size=(1)), #fairness
                    np.random.uniform(low=0, high=1, size=(1)), # degradation
                    np.random.uniform(low=0.5, high=1, size=(1)), # care
                    np.random.uniform(low=0, high=1, size=(1)), # loyalty
                    np.random.uniform(low=0, high=1, size=(1)), # subversion
                    np.random.uniform(low=0.5, high=1, size=(1)), # cheating
                    np.random.uniform(low=0.5, high=1, size=(1)), # harm
                    np.random.uniform(low=0, high=1, size=(1))]).reshape(1,10) #betrayal
    else:
        TypeError('Must be liberal or conservative')
    return poli_init

def number_state(model, state):
    return sum(1 for a in model.grid.get_all_cell_contents() if a.read_state is state)

def number_read(model):
    return number_state(model, State.READ)

def mean_morality(model):
    morality = [a.current_state for a in model.grid.get_all_cell_contents()]
    mean_morality = np.mean(morality, axis = 0)
    return mean_morality
    
class MoralPosts(mesa.Model):
    """A virus model with some number of agents"""

    def __init__(
        self,
        id,
        initial_message,
        G,
        degree,
        echo_party,
    ):
        self.degree = degree
        self.id = id
        self.G = G
        self.initial_message = initial_message
        self.n_iter = 10
        self.iter_counter = 0
        self.grid = mesa.space.NetworkGrid(self.G)
        self.schedule = mesa.time.RandomActivation(self)
        self.echo_party = echo_party
        self.opposition = opposition_party(self.echo_party)
        
        self.datacollector = mesa.DataCollector(
            model_reporters={"Read": number_read,
                             'Mean_morality':mean_morality},
            agent_reporters={"Morals": lambda _: _.current_state,
                             "Degree": lambda _: _.degree,
                             "Party": lambda _: _.party},
        )

        opposition_member = np.random.randint(low=0, high=10, size = None)
        for i, node in enumerate(self.G.nodes()):
            if i == opposition_member:
                a = EmoAgent(
                    i,
                    self,
                    State.UNREAD,
                    gen_political_agent(self.opposition),
                    self.degree[i],
                    self.initial_message,
                    self.opposition
                )
                self.schedule.add(a)
                # Add the agent to the node
                self.grid.place_agent(a, node)            
            else:
                a = EmoAgent(
                    i,
                    self,
                    State.UNREAD,
                    gen_political_agent(self.echo_party),
                    self.degree[i],
                    self.initial_message,
                    self.echo_party
                )
                self.schedule.add(a)
                # Add the agent to the node
                self.grid.place_agent(a, node)
            
        self.running = True
        self.datacollector.collect(self)

        #nodes = list(self.G)
        #angry_nodes = self.random.sample(nodes, 1)
        #for a in self.grid.get_cell_list_contents(angry_nodes):
        #    a.state = State.ANGRY
    
    def new_post(self):
        for a in self.grid.get_cell_list_contents(self.G.nodes()):
            a.read_state = State.UNREAD

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        #self.new_post()

    def run_model(self):
        self.step()

## Run the simulation

In [203]:
final_stats = []
graph_list = GH.G
degree_list = GH.degree

for initial_message in (np.eye(10)):
    initial_message = initial_message.reshape(1, 10)
    initial_morality = []
    final_morality = []
    initial_mean_morality = []
    final_mean_morality = []
    degree_cache = []
    party_cache = []
    for idx in range(ITERATIONS):
        MP = MoralPosts(0, initial_message, G=graph_list[idx], degree = degree_list[idx], echo_party=echo_party)
        MP.run_model()
        # collect results
        mean_moral_states = MP.datacollector.get_model_vars_dataframe()
        initial_mean_morality.append(mean_moral_states.iloc[0].Mean_morality)
        final_mean_morality.append(mean_moral_states.iloc[1].Mean_morality)
        agent_states = pd.DataFrame(MP.datacollector.get_agent_vars_dataframe().to_records())
        initial_morality.append(np.concatenate(agent_states[agent_states.Step == 0]['Morals'].tolist()))
        final_morality.append(np.concatenate(agent_states[agent_states.Step == 1]['Morals'].tolist()))
        degree_cache.append(agent_states[agent_states.Step == 0]['Degree'].tolist())
        party_cache.append(agent_states[agent_states.Step == 0]['Party'].tolist())
    
    initial_morality = np.concatenate(initial_morality)
    final_morality = np.concatenate(final_morality)
    im_pred = lrmodel.predict(initial_morality)
    fn_pred = lrmodel.predict(final_morality)
    degree_cache = [item for sublist in degree_cache for item in sublist]
    party_cache = [item for sublist in party_cache for item in sublist]

    uh = pd.DataFrame({'initial':im_pred, 'final':fn_pred})
    uh['prod'] = uh.final * uh.initial
    uh['changed'] = np.where(uh['prod'] == -1, 1, 0)
    uh['positive_change'] = np.where(uh['changed'] ==uh['final'], 1, 0)
    uh['neg_change'] = 1*(uh.changed != uh.positive_change)
    uh['degree'] = degree_cache
    uh['party'] = party_cache
    uh['echo'] = echo_party
    minority = uh[uh['party'] != uh['echo']].copy()
    minority = minority[minority['changed'] == 1].copy()
    
    final_stats.append({'trials': uh.shape[0],
                        'initial_pos':(uh['initial']==1).sum(),
                        'initial_neg':(uh['initial']==-1).sum(),
                        'final_pos':(uh['final']==1).sum(),
                        'final_neg':(uh['final']==-1).sum(),
                        'changes':uh['changed'].sum(),
                        'pos_changes':uh['positive_change'].sum(),
                        'neg_changes':uh['changed'].sum() - uh['positive_change'].sum(),
                        'pos_correlation':uh['degree'].corr(uh['positive_change']),
                        'neg_correlation':uh['degree'].corr(uh['neg_change']),
                        'minority_pos_changes':minority.positive_change.sum(),
                        'minority_neg_changes':minority.neg_change.sum(),})

/Users/evanwilliams/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/evanwilliams/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/evanwilliams/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/evanwilliams/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/evanwilliams/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticReg

## Export results

In [208]:
out = pd.DataFrame(final_stats)
out['morals'] = moral_categories
out.to_csv('../results/liberal_maj_100k.csv')

In [209]:
out

,trials,initial_pos,initial_neg,final_pos,final_neg,changes,pos_changes,neg_changes,pos_correlation,neg_correlation,minority_pos_changes,minority_neg_changes,morals
0,1000000,423451,576549,614001,385999,234936,212743,22193,-0.107204,0.037859,12215,3102,purity
1,1000000,423517,576483,439231,560769,139888,77801,62087,-0.010993,0.037896,5061,6860,authority
2,1000000,422685,577315,487214,512786,165711,115120,50591,-0.066576,0.043837,11501,4232,fairness
3,1000000,424306,575694,270778,729222,189106,17789,171317,0.025452,-0.042556,1445,14344,degradation
4,1000000,423439,576561,507088,492912,176845,130247,46598,-0.081192,0.043186,13601,3902,care
5,1000000,423529,576471,592703,407297,214848,192011,22837,-0.093691,0.035916,15331,1806,loyalty
6,1000000,423057,576943,293219,706781,175394,22778,152616,0.026915,-0.032399,1773,13037,subversion
7,1000000,423656,576344,291788,708212,187912,28022,159890,0.028520,-0.056449,1150,20679,cheating
8,1000000,423515,576485,281292,718708,195291,26534,168757,0.029877,-0.062642,1073,22308,harm
9,1000000,423424,576576,309840,690160,165172,25794,139378,0.025047,-0.021635,1379,14462,betrayal
